<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Trabajo de fin de máster</p>
<p style="margin: 0; text-align:right;">Máster Universitario en Ciencia de Datos</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
<p style="margin: 0; text-align:right;">Azucena González Muiño</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Construcción del dataset

El conjunto de datos a emplear recoge la información correspondiente a las actuaciones policiales realizadas en Londres de tipo *stop and search*, es decir, aquellas que implican la detención y registro de personas, ya sea como peatones o como conductores de vehículos.

En la web <a href='https://data.police.uk/data/archive/'>data.police.uk</a> pueden obtenerse lotes de ficheros en formato CSV con la información correspondiente a delitos y actuaciones policiales desde finales de 2013 y con licencia de uso <a href='https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/'>OGLv3.0</a>.

Esta descarga histórica no permite aplicar filtros por fecha, por cuerpo de seguridad o por tipo de actuación policial, por lo que es necesario realizar este cribado después de su recolección. El tratamiento realizado en este *notebook* es el siguiente:
<ul>
    <li>Descompresión de los ficheros zip descargados, que deben estar almacenados en la carpeta datasets</li>
    <li>Filtrado de los ficheros pertenecientes a las fuerzas policiales que operan en el Gran Londres: City of London Police, British Transport Police y Metropolitan Police con actuaciones de tipo "stop and search"</li>
    <li>Asignación del origen de datos de cada registro (campo 'Source') que indica el cuerpo policial del que se ha obtenido la información</li>
    <li>Unificación de los registros de diferentes fuentes en un mismo dataset</li>
    <li>Identificación de las distintas localizaciones existentes (latitud y longitud) y obtención de su dirección completa, incluyendo el distrito</li>
    <li>Volcado a fichero de la información de las actuaciones policiales y de las localizaciones con las direcciones completas obtenidas</li>
</ul>

In [1]:
# Interacción con el sistema operativo y la gestión de ficheros y
# directorios
import os
import glob

# Gestión de coordenadas y localizaciones
import geopy.geocoders
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

# Manipulación de datos
import numpy as np
import pandas as pd

# Tratamiento de ficheros comprimidos
from zipfile import ZipFile

In [2]:
# Se selecciona el directorio de trabajo
os.chdir('datasets')

In [3]:
# Función que realiza la descompresión de ficheros zip
def unzip_file(input_file):
    with ZipFile(input_file, 'r') as zip: 
        zip.extractall()

# Se descomprimen los ficheros zip seleccionados del portal de datos de la policía
# y que contienen los datos en formato csv
zip_filenames = [i for i in glob.glob('*.zip')]
for f in zip_filenames:
    unzip_file(f)

In [4]:
# Se seleccionan sólo los ficheros relativos a los cuerpos de policía que operan
# en Londres y se crea una nueva variable que indica el origen (fuerza policial) de los datos

# British Transport Police
btp_filenames = [i for i in glob.glob('*btp-stop-and-search.csv')]
btp_data = pd.concat([pd.read_csv(f) for f in btp_filenames])
btp_data['Source'] = 'BTP'
print('Número de filas (BTP):', btp_data.shape[0])
print('Número de atributos (BTP):', btp_data.shape[1])

# City of London Police
clp_filenames = [i for i in glob.glob('*city-of-london-stop-and-search.csv')]
clp_data = pd.concat([pd.read_csv(f) for f in clp_filenames])
clp_data['Source'] = 'CLP'
print('Número de filas (CLP):', clp_data.shape[0])
print('Número de atributos (CLP):', clp_data.shape[1])

# Metropolitan Police
met_filenames = [i for i in glob.glob('*metropolitan-stop-and-search.csv')]
met_data = pd.concat([pd.read_csv(f) for f in met_filenames])
met_data['Source'] = 'MET'
print('Número de filas (MET):', met_data.shape[0])
print('Número de atributos (MET):', met_data.shape[1])

# Se crea un dataframe con la información de todas las fuerzas policiales
full_data = pd.concat([btp_data, clp_data, met_data], ignore_index=True)
print('Número de filas totales:', full_data.shape[0])
print('Número de atributos:', full_data.shape[1])

# Se revisan los tipos de datos de las columnas
print(full_data.dtypes)

Número de filas (BTP): 28246
Número de atributos (BTP): 16
Número de filas (CLP): 8184
Número de atributos (CLP): 16
Número de filas (MET): 874039
Número de atributos (MET): 16
Número de filas totales: 910469
Número de atributos: 16
Type                                         object
Date                                         object
Part of a policing operation                float64
Policing operation                          float64
Latitude                                    float64
Longitude                                   float64
Gender                                       object
Age range                                    object
Self-defined ethnicity                       object
Officer-defined ethnicity                    object
Legislation                                  object
Object of search                             object
Outcome                                      object
Outcome linked to object of search           object
Removal of more than just outer clothin

In [5]:
# Se cambia el tipo de dato de las variables booleanas
data_conv = {'Part of a policing operation':'boolean',
             'Outcome linked to object of search':'boolean', 
             'Removal of more than just outer clothing':'boolean'}
full_data = full_data.astype(data_conv)
full_data.dtypes

# Se muestran algunos registros para revisar el resultado
full_data.tail()

# Se vuelcan los datos a fichero para su posterior análisis y procesamiento
full_data.to_csv('full-stop-and-search.csv', index=False, float_format='%f')

Para obtener las direcciones exactas a partir de las coordenadas se emplea el servicio gratuito Nominatim. En ocasiones, este sistema puede dar errores o *timeouts*, por lo que posteriormente será preciso añadir de forma manual la dirección de los registros faltantes.

In [6]:
# Se obtienen todas las coordenadas (no repetidas) del conjunto de datos
full_data_locs = (full_data.drop_duplicates(['Latitude','Longitude'])
                  [['Latitude','Longitude']]).dropna()

# Función que devuelve la dirección completa encontrada a partir de las
# coordenadas dadas o None si no posible recuperarla
def get_location(coords):
    try:
        # Se formatea la localización y se solicita su dirección
        coords = ['{:f}'.format(coords[0]), '{:f}'.format(coords[1])]
        address = str(geolocator.reverse(coords))
        return address
    except Exception as exception:
        print(f'Error al tratar las coordenadas {coords}: {exception}')
        return None

# Lista para almacenar las direcciones encontradas
addresses = []

# Se genera un agente para obtener la dirección física a partir de las coordenadas 
geolocator = Nominatim(user_agent="data-police-uk-tfm")

# Se limita la velocidad a la que se solicitarán los datos para evitar saturar
# el servidor y cumplir con las políticas de uso de Nominatim
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Se aumenta el tiempo de espera de respuesta
geopy.geocoders.options.default_timeout = 60

# Se obtienen las direcciones completas
for l in full_data_locs.values.tolist():
    address = get_location(l)
    addresses.append(address)

# Se incorporan al dataframe de localizaciones las direcciones halladas
full_data_locs['Address'] = addresses

Error al tratar las coordenadas ['51.550665', '0.000074']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.455955', '-0.000010']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.568311', '-0.000089']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.560808', '0.000086']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.434553', '-0.000024']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.499272', '0.000071']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.433221', '0.000019']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.456341', '0.000065']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.484075', '0.000040']: HTTP Error 400: Bad Request
Error al tratar las coordenadas ['51.391819', '-0.060462']: Service timed out
Error al tratar las coordenadas ['51.564584', '-0.306623']: Service timed out
Error al tratar las coordenadas ['51.417606', '-0.138385']

In [7]:
# Función para obtener el distrito al que pertenece una localización
def get_borough(full_address):
    # Listado completo de los 33 distritos londinenses
    boroughs = ['City of London', 
                'City of Westminster', 
                'Royal Borough of Kensington and Chelsea',
                'London Borough of Hammersmith and Fulham', 
                'London Borough of Wandsworth',
                'London Borough of Lambeth', 
                'London Borough of Southwark', 
                'London Borough of Tower Hamlets', 
                'London Borough of Hackney',
                'London Borough of Islington', 
                'London Borough of Camden', 
                'London Borough of Brent',
                'London Borough of Ealing', 
                'London Borough of Hounslow',
                'London Borough of Richmond upon Thames', 
                'Royal Borough of Kingston upon Thames',
                'London Borough of Merton', 
                'London Borough of Sutton', 
                'London Borough of Croydon',
                'London Borough of Bromley', 
                'London Borough of Lewisham', 
                'Royal Borough of Greenwich',
                'London Borough of Bexley', 
                'London Borough of Havering', 
                'London Borough of Barking and Dagenham', 
                'London Borough of Redbridge',
                'London Borough of Newham', 
                'London Borough of Waltham Forest', 
                'London Borough of Haringey',
                'London Borough of Enfield', 
                'London Borough of Barnet', 
                'London Borough of Harrow', 
                'London Borough of Hillingdon']
    
    # Se busca el distrito 
    try:
        for borough in boroughs:
            if borough in full_address:
                # Se devuelve el nombre corto del distrito
                if borough == 'City of London':
                    return borough
                else:
                    return borough[borough.index('of')+3:len(borough)]
    except:
        return None

# Se incorporan al dataframe de localizaciones los distritos
full_data_locs['Borough'] = full_data_locs['Address'].apply(get_borough)

# Se muestran algunos registros para comprobar el resultado
full_data_locs.tail()

,Latitude,Longitude,Address,Borough
910119,51.343241,-0.108390,"Brighton Road / Purley Downs Road, Brighton Ro...",Croydon
910132,51.506104,-0.238069,"Coningham Mews, Brook Green, London Borough of...",Hammersmith and Fulham
910147,51.449536,0.098686,"Lime Grove, Blackfen, London Borough of Bexley...",Bexley
910285,51.528617,-0.296871,"Brunswick Gardens, Park Royal, London Borough ...",Ealing
910374,51.580182,-0.145807,"38, Summersby Road, Shepherd's Hill, Highgate,...",Haringey


In [8]:
# Se vuelcan los datos a fichero para su posterior análisis y procesamiento
full_data_locs.to_csv('full-stop-and-search-locs.csv', index=False, float_format='%f')